In [1]:
#https://towardsdatascience.com/deep-transfer-learning-for-natural-language-processing-text-classification-with-universal-1a2c69e5baa9
import os, pandas as pd
import re
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np

In [2]:
file_list=os.listdir('../../dataset/df_train.pkl.gz/')
df_train=pd.DataFrame()
for file in file_list:
    df_train=pd.concat([df_train, 
                        pd.read_pickle('../../dataset/df_train.pkl.gz/'+file, compression='gzip')])
df_train['mission_prgrm']=df_train['mission']+'; '+df_train['prgrm_dsc']

small_num=0
while small_num<100: # Make sure each category has at least 100 records.
    trainDF = df_train[df_train.mission.notna() & df_train.NTEE1.notna()].sample(40000)
    small_num=trainDF.groupby('NTEE1').count().sort_values('EIN').iloc[0]['EIN']
    
#### Sample ####
#len(train_df)

In [3]:
df_train['labels'] = df_train['NTEE1'].apply(lambda x:ord(x)-65)
df_train.labels.head()

592785    10
592787    10
592789    13
592797    11
592800    14
Name: labels, dtype: int64

In [4]:
texts = df_train['mission_prgrm'].values
labels = df_train['labels'].values

train_texts = texts[:3500]
train_labels = labels[:3500]

val_texts = texts[3500:4000]
val_labels = labels[3500:4000]

test_texts = texts[4000:]
test_labels = labels[4000:]

train_texts.shape, val_texts.shape, test_texts.shape

((3500,), (500,), (225472,))

In [5]:
#text-preprocessing: Eliminating steps for contractions (We've --> We have)

import contractions
from bs4 import BeautifulSoup
import unicodedata
import re

def strip_html_tags(text):
    soup = BeautifulSoup(text, "html.parser")
    [s.extract() for s in soup(['iframe', 'script'])]
    stripped_text = soup.get_text()
    stripped_text = re.sub(r'[\r|\n|\r\n]+', '\n', stripped_text)
    return stripped_text

def remove_accented_chars(text):
    text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    return text

def expand_contractions(text):
    return contractions.fix(text)

def remove_special_characters(text, remove_digits=False):
    pattern = r'[^a-zA-Z0-9\s]' if not remove_digits else r'[^a-zA-Z\s]'
    text = re.sub(pattern, '', text)
    return text

def pre_process_document(document):
    # strip HTML
    #document = strip_html_tags(document)
    # lower case
    document = document.lower()
    # remove extra newlines (often might be present in really noisy text)
    document = document.translate(document.maketrans("\n\t\r", "   "))
    # remove accented characters
    #document = remove_accented_chars(document)
    # expand contractions    
    #document = expand_contractions(document)  
    # remove special characters and\or digits    
    # insert spaces between special characters to isolate them    
    special_char_pattern = re.compile(r'([{.(-)!}])')
    document = special_char_pattern.sub(" \\1 ", document)
    document = remove_special_characters(document, remove_digits=True)  
    # remove extra whitespace
    #document = re.sub(' +', ' ', document)
    document = document.strip()
    
    return document


pre_process_corpus = np.vectorize(pre_process_document)

train_texts = pre_process_corpus(train_texts)
val_texts = pre_process_corpus(val_texts)
test_texts = pre_process_corpus(test_texts)

MemoryError: 

In [6]:
# Training input on the whole training set with no limit on training epochs.
train_input_fn = tf.estimator.inputs.numpy_input_fn(
    {'sentence': train_texts}, train_labels, 
    batch_size=256, num_epochs=None, shuffle=True)
    
# Prediction on the whole training set.
predict_train_input_fn = tf.estimator.inputs.numpy_input_fn(
    {'sentence': train_texts}, train_labels, shuffle=False)
    
# Prediction on the whole validation set.
predict_val_input_fn = tf.estimator.inputs.numpy_input_fn(
    {'sentence': val_texts}, val_labels, shuffle=False)
    
# Prediction on the test set.
predict_test_input_fn = tf.estimator.inputs.numpy_input_fn(
    {'sentence': test_texts}, test_labels, shuffle=False)  

In [12]:
embedding_feature = hub.text_embedding_column(
    key='sentence', 
    module_spec="https://tfhub.dev/google/universal-sentence-encoder/2",
    trainable=False)

In [17]:
'''
dnn = tf.estimator.DNNClassifier(
          hidden_units=[512, 128],
          feature_columns=[embedding_feature],
          n_classes=26,
          activation_fn=tf.nn.relu,
          dropout=0.1,
          optimizer=tf.train.AdagradOptimizer(learning_rate=0.005))

#73.6

dnn = tf.estimator.DNNClassifier(
          hidden_units=[512, 128],
          feature_columns=[embedding_feature],
          n_classes=26,
          activation_fn=tf.nn.relu,
          dropout=0.3,
          optimizer=tf.train.AdagradOptimizer(learning_rate=0.01))
#75

dnn = tf.estimator.DNNClassifier(
          hidden_units=[512, 128],
          feature_columns=[embedding_feature],
          n_classes=26,
          activation_fn=tf.nn.relu,
          dropout=0.3,
          optimizer=tf.train.AdagradOptimizer(learning_rate=0.02))
#75.2

'''
dnn = tf.estimator.DNNClassifier(
          hidden_units=[512, 128],
          feature_columns=[embedding_feature],
          n_classes=26,
          activation_fn=tf.nn.elu,
          dropout=0.3,
          optimizer=tf.train.AdagradOptimizer(learning_rate=0.02))


In [18]:
tf.logging.set_verbosity(tf.logging.ERROR)
import time

TOTAL_STEPS = 1500   #1500
STEP_SIZE = 100   #100
for step in range(0, TOTAL_STEPS+1, STEP_SIZE):
    print()
    print('-'*100)
    print('Training for step =', step)
    start_time = time.time()
    dnn.train(input_fn=train_input_fn, steps=STEP_SIZE)
    elapsed_time = time.time() - start_time
    print('Train Time (s):', elapsed_time)
    print('Eval Metrics (Train):', dnn.evaluate(input_fn=predict_train_input_fn))
    print('Eval Metrics (Validation):', dnn.evaluate(input_fn=predict_val_input_fn))


----------------------------------------------------------------------------------------------------
Training for step = 0
Train Time (s): 161.90317130088806
Eval Metrics (Train): {'accuracy': 0.76257145, 'average_loss': 0.7984377, 'loss': 99.80471, 'global_step': 100}
Eval Metrics (Validation): {'accuracy': 0.742, 'average_loss': 0.9658379, 'loss': 120.72974, 'global_step': 100}

----------------------------------------------------------------------------------------------------
Training for step = 100
Train Time (s): 206.26475620269775
Eval Metrics (Train): {'accuracy': 0.80057144, 'average_loss': 0.6698809, 'loss': 83.735115, 'global_step': 200}
Eval Metrics (Validation): {'accuracy': 0.748, 'average_loss': 0.9680547, 'loss': 121.006836, 'global_step': 200}

----------------------------------------------------------------------------------------------------
Training for step = 200
Train Time (s): 188.30295634269714
Eval Metrics (Train): {'accuracy': 0.8165714, 'average_loss': 0.609

# 